In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, Dict

import warnings
warnings.filterwarnings('ignore')

In [2]:
class SimpleTimeSeriesDataset(Dataset):
    def __init__(self, X, y, seq_length):  # seq_length: n개월 간
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.seq_length = seq_length

    def __len__(self):
        return len(self.X) - self.seq_length

    def __getitem__(self, idx):
        return (self.X[idx:idx+self.seq_length], self.y[idx+self.seq_length])

In [3]:
class LSTM_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers=2, dropout=0.2, output_dim=1):
        super().__init__()
        self.batch_norm = nn.BatchNorm1d(input_dim)

        # Bidirectional LSTM
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0,
            bidirectional=True
        )

        # 시퀀스 가중치
        self.seq_weight = nn.Linear(hidden_dim * 2, 1)

        # FC layer
        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, x):
        batch_size = x.size(0)
        seq_len = x.size(1)

        # batch normalization으로 입력 처리
        x = x.view(-1, x.size(-1))
        x = self.batch_norm(x)
        x = x.view(batch_size, seq_len, -1)

        # LSTM 실행
        lstm_out, _ = self.lstm(x)  # 출력 크기: [batch_size, seq_len, hidden_dim * 2]; 양방향이기 때문에 출력이 두 개

        # 시퀀스 가중치 계산
        seq_weights = torch.softmax(self.seq_weight(lstm_out), dim=1)  # softmax: 가중치 정규화 목적
        output = torch.sum(seq_weights * lstm_out, dim=1)  # 가중 합계로 시퀀스를 하나의 벡터로 압축

        # FC layers
        output = self.fc_layers(output)

        return output

In [4]:
# Mean Absolute Percentage Error 계산

def calculate_mape(y_true: np.ndarray, y_pred: np.ndarray) -> Dict:
    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # 평균 예측값 사용 (3개월 예측 평균)
    if y_pred.ndim > 1:
        y_pred = np.mean(y_pred, axis=1)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
def prepare_data(train, test):
    # 데이터 불러오기
    train_data = pd.read_csv(train)
    test_data = pd.read_csv(test)

    # 데이터의 날짜 범위 확인
    print("Training data date range:", train_data['date'].iloc[0], "to", train_data['date'].iloc[-1])
    print("Test data date range:", test_data['date'].iloc[0], "to", test_data['date'].iloc[-1])

    # 날짜 처리
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    # # 시계열 특성 추가 (feature engineering)
    # 1. 계절성(월별) 특성
    train_data['month'] = train_data['date'].dt.month
    test_data['month'] = test_data['date'].dt.month

    # 2. 이동평균 특성
    for window in [3, 6, 9, 12]:  # 3, 6, 12개월 동안의 가격 이동 평균 (moving average)
        train_data[f'price_ma{window}'] = train_data['Natural_Gas_US_Henry_Hub_Gas'].rolling(window=window).mean()
        test_data[f'price_ma{window}'] = test_data['Natural_Gas_US_Henry_Hub_Gas'].rolling(window=window).mean()

    # 3. 가격 변화율
    train_data['price_change'] = train_data['Natural_Gas_US_Henry_Hub_Gas'].pct_change()  # (현재 가격 - 이전 가격) / 이전 가격
    test_data['price_change'] = test_data['Natural_Gas_US_Henry_Hub_Gas'].pct_change()

    # 4. 변동성 (표준편차)
    for window in [3, 6, 9, 12]:
        train_data[f'price_volatility_{window}'] = train_data['Natural_Gas_US_Henry_Hub_Gas'].rolling(window=window).std()
        test_data[f'price_volatility_{window}'] = test_data['Natural_Gas_US_Henry_Hub_Gas'].rolling(window=window).std()

    # 새로 만든 특성만 저장한 파일 생성
    new_features = ['date', 'month', 'price_ma3', 'price_ma6', 'price_ma9', 'price_ma12', 'price_change', 'price_volatility_3', 'price_volatility_6', 'price_volatility_9', 'price_volatility_12']

    new_train_data = train_data[new_features].copy()
    new_train_data.to_csv('new_data_train.csv', index=False)

    new_test_data = test_data[new_features].copy()
    new_test_data.to_csv('new_data_test.csv', index=False)


    # 특징
    feature_cols = [
        # 에너지 가격 변수
        'Brent_Crude_Oil', 'WTI_Crude_Oil', 'Electricity_Price_USA',

        # 공급과 수요
        'Natural_Gas_Imports_From_Canada_USA', 'Natural_Gas_Imports_USA',
        'Natural_Gas_Total_Consumption_USA', 'Natural_Gas_Rotary_Rig_Count_USA',
        'Total_Natural_Gas_Marketed_Production_USA', 'Total_Natural_Gas_Underground_Storage_Volume_USA',

        # 소비자 물가 지수
        'CPI_Energy_Seasonally_Adjusted_USA', 'CPI_Index_Seasonally_Adjusted_USA',

        # 블룸버그 천연 가스 , 미국 달러 지수
        'BCOMNG_INDX', 'DXY_INDX',

        # feature engineering
        'price_ma3', 'price_ma6', 'price_ma9', 'price_ma12', 'price_change', 'price_volatility_3', 'price_volatility_6', 'price_volatility_9', 'price_volatility_12',

        # 추가 변수
        'FEDFUNDS', 'INDPRO', 'ONI',  # 'GPR', 'GPRHC_RUS', 'GPRHC_UKR', 'GPRHC_USA', 'GPRH', 'VIXCLS', 'GDP'
    ]

    # 측정할 값
    target_col = 'Natural_Gas_US_Henry_Hub_Gas'

    # 결측치 처리
    for col in feature_cols + [target_col]:
        train_data[col] = train_data[col].fillna(method='ffill').fillna(method='bfill')
        test_data[col] = test_data[col].fillna(method='ffill').fillna(method='bfill')

    # 특징과 타겟 분리
    X_train = train_data[feature_cols].values
    y_train = train_data[target_col].values
    X_test = test_data[feature_cols].values
    y_test = test_data[target_col].values

    # 스케일링
    scaler_X = RobustScaler()
    scaler_y = RobustScaler()

    X_train_scaled = scaler_X.fit_transform(X_train)
    y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
    X_test_scaled = scaler_X.transform(X_test)

    return X_train_scaled, y_train_scaled, X_test_scaled, scaler_y, test_data['date'].values, y_test

In [6]:
# 모델 학습

def train_model(X_train, y_train, n_epoch, seq_length) -> Tuple[LSTM_model, Dict]:
    # 데이터셋 생성

    train_dataset = SimpleTimeSeriesDataset(X_train, y_train, seq_length=seq_length)
    train_size = int(len(train_dataset) * 0.8)
    train_dataloader = DataLoader(torch.utils.data.Subset(train_dataset, range(train_size)), batch_size=32, shuffle=True)


    X_valid = X_train[train_size:]
    y_valid = y_train[train_size:]

    valid_dataloader = None
    if len(X_valid) > 6:  # check for valid length of the validation set
      valid_dataset = SimpleTimeSeriesDataset(X_valid, y_valid, seq_length=seq_length)
      valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


    # 모델 초기화
    model = LSTM_model(input_dim=X_train.shape[1], hidden_dim=256, n_layers=4, dropout=0.3)

    # 손실 함수와 옵티마이저
    criterion = nn.HuberLoss(delta=1.0)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # 학습 기록
    history = {'train_loss': [], 'valid_loss': []}

    best_valid_loss = float('inf')
    patience = 10
    patience_counter = 0

    # 학습
    model.train()
    for epoch in range(n_epoch):
        total_loss = 0
        for X_batch, y_batch in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch.view(-1, 1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        history['train_loss'].append(avg_loss)

        # validation
        if valid_dataloader: # check if validation loader is defined
            model.eval()
            total_valid_loss = 0
            with torch.no_grad():
              for X_batch, y_batch in valid_dataloader:
                y_pred = model(X_batch)
                valid_loss = criterion(y_pred, y_batch.view(-1, 1))
                total_valid_loss += valid_loss.item()

            avg_valid_loss = total_valid_loss / len(valid_dataloader)
            history['valid_loss'].append(avg_valid_loss)

            scheduler.step(avg_valid_loss)

            # Early stoptting
            if avg_valid_loss < best_valid_loss:
              best_valid_loss = avg_valid_loss
              patience_counter = 0
            else:
              patience_counter += 1

            if patience_counter >= patience:
              print(f'Early stopping at epoch {epoch+1}')
              break

        if (epoch + 1) % 50 == 0:
            print(f'Epoch [{epoch+1}/{n_epoch}], Training Loss: {avg_loss:.4f}, Valid Loss: {avg_valid_loss:.4f}')

    # Loss 그래프
    plt.figure(figsize=(10, 6))
    plt.plot(history['train_loss'], label='Train Loss')

    if 'valid_loss' in history:  # if validation loss was recorded
        plt.plot(history['valid_loss'], label='Validation Loss')

    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('training_history.png')
    plt.close()

    return model, history

In [7]:
# 앙상블

def train_ensemble_models(X_train, y_train, n_epochs, sequence_lengths=[3, 6, 12]):
    models = {}
    histories = {}

    # 각 시퀀스 길이별로 모델 학습
    for seq_len in sequence_lengths:
        print(f"\nTraining model with sequence length {seq_len}")
        model, history = train_model(X_train, y_train, n_epochs, seq_len)  # seq_length 파라미터 추가 필요
        models[seq_len] = model
        histories[seq_len] = history

    return models, histories

def ensemble_predictions(models, X_test, scaler_y, weights=None):
    all_predictions = []

    # 각 시점에 대해
    for i in range(len(X_test)):
        model_predictions = []

        # 각 모델의 예측값 얻기
        for seq_len, model in models.items():
            # 시퀀스 길이에 따른 패딩 처리
            if i < seq_len - 1:
                padding = np.tile(X_test[0], (seq_len - 1 - i, 1))
                current_seq = np.vstack([padding, X_test[:i + 1]])
            else:
                current_seq = X_test[i - seq_len + 1:i + 1]

            input_seq = torch.FloatTensor(current_seq).unsqueeze(0)
            predictions = []

            # 테스트 데이터의 월별 3개월 예측
            for _ in range(3):
                pred = model(input_seq)
                scaled_pred = scaler_y.inverse_transform(pred.detach().numpy())[0]
                predictions.append(scaled_pred[0])

                current_seq = np.vstack([current_seq[1:], current_seq[-1]])
                input_seq = torch.FloatTensor(current_seq).unsqueeze(0)

            model_predictions.append(predictions)

        # 앙상블 (가중 평균)
        if weights is None:
            weights = [1/len(models)] * len(models)  # 동일 가중치

        ensemble_pred = np.average(model_predictions, axis=0, weights=weights)
        all_predictions.append(ensemble_pred)

    return all_predictions

In [8]:
# 예측 결과 분석

def analyze_predictions(y_true: np.ndarray, y_pred: np.ndarray, dates: np.ndarray) -> Dict:
    # 길이 확인 및 조정
    min_len = min(len(y_true), len(y_pred), len(dates))

    y_true = y_true[:min_len]
    y_pred = y_pred[:min_len]
    dates = dates[:min_len]

    # 평균 예측값 사용 (3개월 예측 평균)
    if y_pred.ndim > 1:
        y_pred = np.mean(y_pred, axis=1)

    # 메트릭 계산
    metrics = {'mape': calculate_mape(y_true, y_pred)} # return mape as a dictionary

    # 오차 분포 분석
    errors = y_pred - y_true
    error_stats = {
        'mean_error': np.mean(errors),
        'std_error': np.std(errors),
        'max_error': np.max(np.abs(errors)),
        'error_distribution': errors
    }

    # 시각화
    plt.figure(figsize=(15, 10))

    # 1. 실제 vs 예측 가격
    plt.subplot(2, 2, 1)
    plt.plot(dates, y_true, label='Actual')
    plt.plot(dates, y_pred, label='Predicted')
    plt.title('Actual vs Predicted Prices')
    plt.legend()

    # 2. 오차 분포
    plt.subplot(2, 2, 2)
    sns.histplot(errors, bins=30)
    plt.title('Error Distribution')

    # 3. 예측 vs 실제 가격 산점도
    plt.subplot(2, 2, 3)
    plt.scatter(y_true, y_pred)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    plt.title('Predicted vs Actual')

    plt.tight_layout()
    plt.savefig('prediction_analysis.png')
    plt.close()

    return {'metrics': metrics, 'error_stats': error_stats}

In [9]:
def main():
    # 데이터 준비
    X_train, y_train, X_test, scaler_y, test_dates, y_test = prepare_data('data_train_new.csv', 'data_test_new.csv')

    # 앙상블 모델 학습
    sequence_lengths = [3, 6, 9, 12]  # 3, 6, 9, 12개월;
    models, histories = train_ensemble_models(X_train, y_train, 100, sequence_lengths)

    # 앙상블 예측
    weights = [0.1, 0.2, 0.3, 0.4]  # 옵션: [0.1, 0.2, 0.3, 0.4], [0.15, 0.2, 0.3, 0.35]; [0.1, 0.15, 0.35, 0.4]
    all_predictions = ensemble_predictions(models, X_test, scaler_y, weights)

    # 예측 분석
    test_predictions = np.array(all_predictions)[:-2]  # 마지막 두 개 제외 (3개월 예측 때문)
    y_test_subset = y_test[:len(test_predictions)]
    dates_subset = test_dates[:len(test_predictions)]
    analysis_results = analyze_predictions(y_test_subset, test_predictions, dates_subset)

    print(f"MAPE: {analysis_results['metrics']['mape']:.2f}%")

    # 제출 파일 생성
    dates = pd.to_datetime(test_dates)
    submission_data = []

    # 테스트 데이터의 각 날짜에 대해 예측값 저장
    for date, preds in zip(dates, all_predictions):
        if preds[0] is not None:  # 예측이 가능한 경우만 포함
            submission_data.append({
                'date': date.strftime('%Y-%m-%d'),
                'pred_1': preds[0],
                'pred_2': preds[1],
                'pred_3': preds[2]
            })

    submission = pd.DataFrame(submission_data)
    submission.to_csv('submission.csv', index=False)
    print("\nSubmission file shape:", submission.shape)
    print("First date in submission:", submission['date'].iloc[0])
    print("Last date in submission:", submission['date'].iloc[-1])
    print("Predictions saved to submission.csv")

if __name__ == "__main__":
    main()

Training data date range: 2003-09-30 to 2020-08-31
Test data date range: 2020-09-30 to 2024-08-31

Training model with sequence length 3
Epoch [50/100], Training Loss: 0.0125, Valid Loss: 0.0033
Epoch [100/100], Training Loss: 0.0117, Valid Loss: 0.0032

Training model with sequence length 6
Epoch [50/100], Training Loss: 0.0105, Valid Loss: 0.0031
Epoch [100/100], Training Loss: 0.0103, Valid Loss: 0.0030

Training model with sequence length 9
Epoch [50/100], Training Loss: 0.0089, Valid Loss: 0.0031
Epoch [100/100], Training Loss: 0.0080, Valid Loss: 0.0031

Training model with sequence length 12
Epoch [50/100], Training Loss: 0.0086, Valid Loss: 0.0039
Epoch [100/100], Training Loss: 0.0084, Valid Loss: 0.0039
MAPE: 22.52%

Submission file shape: (48, 4)
First date in submission: 2020-09-30
Last date in submission: 2024-08-31
Predictions saved to submission.csv
